# **Principal component analysis (PCA) Pipeline**


loading libraries and data preparation

In [ ]:
# Load necessary libraries
library(DESeq2)
library(ggplot2)
library(data.table)
# Assuming `dds` is your existing DESeq2 object
# Load the data
clinical_data <- fread("cleaned_clinical_data.csv")

# Step 1: Ensure that the clinical data aligns with the sample names in the DESeq2 object
# Reorder clinical_data to match the column names of count data in dds
clinical_data_ordered <- clinical_data[match(colnames(dds), clinical_data$case_submitter_id),]

# Step 2: Check for any missing or unmatched samples
if(any(is.na(clinical_data_ordered$case_submitter_id))) {
  stop("Some samples in the count data do not match the 'case_submitter_id' in clinical data.")
}

# Step 3: Add the new metadata columns to the DESeq2 object
colData(dds)$age_at_index <- clinical_data_ordered$age_at_index
colData(dds)$gender <- clinical_data_ordered$gender
colData(dds)$ethnicity <- clinical_data_ordered$ethnicity


Data transformation and PCA Calculation

In [ ]:
# Step 4: Transform the data using vst (or rlog if preferred)
vst_data <- vst(dds, blind = TRUE)

# Extract the assay data from the transformed DESeq2 object for PCA
pcaData <- plotPCA(vst_data, intgroup = c("Condition", "age_at_index", "gender", "ethnicity", "race"), returnData = TRUE)

# Calculate percentage of variance explained by PC1 and PC2
percentVar <- round(100 * attr(pca_data, "percentVar"))
# Convert 'Age' to categorical with two groups: "Above 60" and "Under 60"
pcaData$AgeGroup <- ifelse(pcaData$Age > 60, "Above 60", "Under 60")

# Create a shape variable for 'Condition', excluding NA values
pcaData <- pcaData[!is.na(pcaData$Condition), ]  # Remove NA values for Condition
pcaData$Shape <- ifelse(pcaData$Condition == "N_A", 16, 17)  # 16 for circles, 17 for triangles

# Ensure that 'Shape' and other grouping variables are factors for ggplot
pcaData$Shape <- factor(pcaData$Shape, levels = c(16, 17), labels = c("N_A", "H_E"))
pcaData$Gender <- factor(pcaData$Gender)
pcaData$Ethnicity <- factor(pcaData$Ethnicity)
pcaData$Race <- factor(pcaData$Race)
pcaData$AgeGroup <- factor(pcaData$AgeGroup)

**Plotting**

In [ ]:
# Update color palettes based on your specific request
age_colors <- c("#F8766D", "#00BFC4")  # Two levels: Above 60, Under 60
gender_colors <- c("#F8766D", "#00BFC4")  # Two levels: Female, Male
ethnicity_colors <- c("#F8766D", "#00BFC4", "gray50", "green")  # Ethnicity levels
race_colors <- c("green", "#F8766D", "gray50", "orange", "#00BFC4")  # Race levels

# Wrap long legend titles to prevent overlap
levels(pcaData$Race) <- str_wrap(levels(pcaData$Race), width = 15)  # Increase wrap width to 15 characters per line

# Function to extract legends from ggplot
get_legend <- function(my_plot) {
  tmp <- ggplotGrob(my_plot)
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}

# Create the PCA plot colored by AgeGroup without shape legend
p1 <- ggplot(pcaData, aes(x = PC1, y = PC2, color = AgeGroup)) +
  geom_point(aes(shape = Shape), size = 3) +
  scale_color_manual(values = age_colors) +
  scale_shape_manual(values = c(16, 17), guide = "none") +  # Remove shape legend
  coord_fixed() +  # Fix aspect ratio to ensure square plots
  theme_minimal(base_size = 15) +
  theme(panel.background = element_rect(fill = "gray85"),
        legend.position = "right",
        legend.margin = margin(5, 5, 5, 5)) +  # Adjust legend margin
  labs(title = "a", x = "PC1: 20% variance", y = "PC2: 9% variance")

# Create the PCA plot colored by Gender without shape legend
p2 <- ggplot(pcaData, aes(x = PC1, y = PC2, color = Gender)) +
  geom_point(aes(shape = Shape), size = 3) +
  scale_color_manual(values = gender_colors) +
  scale_shape_manual(values = c(16, 17), guide = "none") +  # Remove shape legend
  coord_fixed() +  # Fix aspect ratio to ensure square plots
  theme_minimal(base_size = 15) +
  theme(panel.background = element_rect(fill = "gray85"),
        legend.position = "right",
        legend.margin = margin(5, 5, 5, 5)) +  # Adjust legend margin
  labs(title = "b", x = "PC1: 20% variance", y = "PC2: 9% variance")

# Create the PCA plot colored by Ethnicity without shape legend
p3 <- ggplot(pcaData, aes(x = PC1, y = PC2, color = Ethnicity)) +
  geom_point(aes(shape = Shape), size = 3) +
  scale_color_manual(values = ethnicity_colors) +  # Ensure enough colors for all levels
  scale_shape_manual(values = c(16, 17), guide = "none") +  # Remove shape legend
  coord_fixed() +  # Fix aspect ratio to ensure square plots
  theme_minimal(base_size = 15) +
  theme(panel.background = element_rect(fill = "gray85"),
        legend.position = "right",
        legend.margin = margin(5, 5, 5, 5)) +  # Adjust legend margin
  labs(title = "c", x = "PC1: 20% variance", y = "PC2: 9% variance")

# Create the PCA plot colored by Race without shape legend
p4 <- ggplot(pcaData, aes(x = PC1, y = PC2, color = Race)) +
  geom_point(aes(shape = Shape), size = 3) +
  scale_color_manual(values = race_colors) +  # Ensure enough colors for all levels
  scale_shape_manual(values = c(16, 17), guide = "none") +  # Remove shape legend
  coord_fixed() +  # Fix aspect ratio to ensure square plots
  theme_minimal(base_size = 15) +
  theme(panel.background = element_rect(fill = "gray85"),
        legend.position = "right",
        legend.margin = margin(5, 5, 5, 5),
        legend.key.height = unit(1.8, "lines")) +  # Increase space between legend items
  labs(title = "d", x = "PC1: 20% variance", y = "PC2: 9% variance")

# Extract legends for color only
legend1 <- get_legend(p1)
legend2 <- get_legend(p2)
legend3 <- get_legend(p3)
legend4 <- get_legend(p4)

# Combine plots and legends using cowplot
combined_plot <- plot_grid(
  plot_grid(p1 + theme(legend.position = "none"), legend1, ncol = 2, rel_widths = c(3, 1)),
  plot_grid(p2 + theme(legend.position = "none"), legend2, ncol = 2, rel_widths = c(3, 1)),
  plot_grid(p3 + theme(legend.position = "none"), legend3, ncol = 2, rel_widths = c(3, 1)),
  plot_grid(p4 + theme(legend.position = "none"), legend4, ncol = 2, rel_widths = c(3, 1)),
  ncol = 2
)

# Create a separate plot for the shape legend only
shape_legend_plot <- ggplot(pcaData, aes(x = PC1, y = PC2, shape = Shape)) +
  geom_point(size = 3) +
  scale_shape_manual(values = c(16, 17),
                     labels = c("N_A", "H_E"),
                     name = "Condition") +  # Correct legend title
  theme_void() +  # No axis or background
  theme(legend.position = "bottom",
        legend.box.just = "center",
        legend.margin = margin(t = 0, unit = 'cm'),
        legend.text = element_text(size = 14),  # Increase font size for legend text
        legend.title = element_text(size = 16, face = "bold")) +  # Increase font size for legend title
  guides(shape = guide_legend(nrow = 1, byrow = TRUE))

# Extract the shape legend
shape_legend_grob <- get_legend(shape_legend_plot)

# Combine all plots with the combined shape legend at the bottom
final_plot <- plot_grid(
  combined_plot,
  shape_legend_grob,  # Add the shape legend at the bottom
  ncol = 1,
  rel_heights = c(4, 0.3)  # Adjust heights to allow space for the legend
)

# Display the final plot with the enlarged shape legend at the bottom
final_plot

